In [3]:
import pymongo

myclient = pymongo.MongoClient("mongodb://localhost:27017/")

logdb = myclient["logdb"]
mycol = logdb["logs"]


In [5]:
registros = [
    { "timestamp": '2024-06-05 10:00', "texto": 'texto 1' },
    { "timestamp": '2024-06-05 11:00', "texto": 'texto 2' }
]
mycol.insert_many(registros)

InsertManyResult([ObjectId('6660caca1cd44d0bf65299e0'), ObjectId('6660caca1cd44d0bf65299e1')], acknowledged=True)

In [6]:
print(logdb.list_collection_names())

['logs']


In [7]:
print(myclient.list_database_names())

['admin', 'config', 'local', 'logdb']


In [3]:
x = mycol.delete_many({})
print (x)

result = mycol.find()

#print the result:
for x in result:
  print(x)

DeleteResult({'n': 74161666, 'ok': 1.0}, acknowledged=True)


In [12]:
myquery = { "descricao": {"$regex": "^.+$"} }
x = mycol.delete_many(myquery)

print(x.deleted_count, " documents deleted.")

1  documents deleted.


In [30]:
from pathlib import Path 

arquivo = r'/home/ricardo/pucrs/TCC/logs-node1-b2b/archive.log.D20240331.T033337'
print(Path(arquivo).name)

archive.log.D20240331.T033337


## Apaga arquivos desnecessarios

In [ ]:
import glob, os
for f in glob.glob("failed_deleteP*.jpg"):
    os.remove(f)

## Lista arquivos de log em pasta, com recursividade

In [1]:
import glob

pasta = r'/home/ricardo/pucrs/TCC/logs-node1-b2b/**/*.log*'
lista_arqs = glob.glob(pasta, recursive = True)
#print(lista_arqs)
print(len(lista_arqs))


180281


In [4]:
import re
a = '[2024-03-31 03:21:51.234] ALL '
a += 'Vou colocar um texto de teste '
print("string=",a)

re_comp = re.compile('^\[(\d{4}-\d{2}-\d{2}\s\d{2}:\d{2}:\d{2}\.\d+)\](.+)$', re.S)

groups = re_comp.search(a)
print ('group1:',groups.group(1))
print ('group2:',groups.group(2))


string= [2024-03-31 03:21:51.234] ALL Vou colocar um texto de teste 
group1: 2024-03-31 03:21:51.234
group2:  ALL Vou colocar um texto de teste 


In [26]:
import re, os
path = '/home/ricardo/pucrs/TCC/logs-node1-b2b/LOGS_01_10_2023_32842_83/OdetteFTP.log.XXX'
nomearq = os.path.basename(path).split('/')[-1]
print(nomearq)

re_comp = re.compile('^(.+\.log)(\.)?(.*)$')

#re_comp = re.compile('^\[(\d{4}-\d{2}-\d{2}\s\d{2}:\d{2}:\d{2}\.\d+)\](.+)$', re.S)

groups = re_comp.search(nomearq)
print ('group1:',groups.group(1))
print ('group3:',groups.group(3))


OdetteFTP.log.XXX
group1: OdetteFTP.log
group3: XXX


In [9]:
x = '/home/ricardo/pucrs/TCC/logs-node1-b2b/archive.log.D20240331.T033337'
x = 'archive.log.D20240331.T033337'
print(os.path.basename(x).split('/')[-1])

archive.log.D20240331.T033337


## Ler cada arquivo e grava cada registro na base de dados

In [43]:
import re
import os
from tqdm import tqdm
from pathlib import Path 
import glob
import pymongo
import datetime

myclient = pymongo.MongoClient("mongodb://localhost:27017/")

logdb = myclient["logdb"]
mycol = logdb["logs"]
pasta = r'/home/ricardo/pucrs/TCC/logs-node1-b2b/**/*.log*'
#pasta = r'/home/ricardo/pucrs/TCC/log-teste/**/*.log*'
arqs = glob.glob(pasta, recursive = True)
print("Quantidade de arquivos de log:",len(arqs))

# deleta todas as linhas de log
_ = mycol.delete_many({})
print (_)

registros_log = []

regex_registro = re.compile('^\[(\d{4}-\d{2}-\d{2}\s\d{2}:\d{2}:\d{2}\.\d+)\](.+)$', re.S)
regex_loop     = re.compile('^\[(\d{4}-\d{2}-\d{2}\s\d{2}:\d{2}:\d{2}\.\d+)\]')
regex_nmlog    = re.compile('^(.+\.log)(\.)?(.*)$')

def gravaRegistro(linha, path):
    global registros_log
    timestamp = ''
    texto = '' 
    # Recupera data/hora e texto do log
    groups = regex_registro.search(linha)
    if groups:
        timestamp = groups.group(1)
        texto = groups.group(2)

    if (timestamp and texto):
        #
        # Separa o nome do log de sua data/hora
        nomearq1 = os.path.basename(path).split('/')[-1]
        groups   = regex_nmlog.search(nomearq1)
        nomearq  = groups.group(1)
        datahora = groups.group(3)
        #
        dt = datetime.datetime.strptime(timestamp, "%Y-%m-%d %H:%M:%S.%f")        
        registros_log.append({"arq_nome": nomearq, 'arq_dthr': datahora, 'path': path, "timestamp": dt, "texto": texto})
        if (len(registros_log) == 20000):
            mycol.insert_many(registros_log)
            registros_log = []
    return

for arq in tqdm(arqs):
    registro = ''
    if (os.path.isfile(arq)):
        with open(arq,'r',encoding='iso-8859-1') as fh:
            nrlinha = 0
            try:
                for linha in fh:
                    if (len(registro) > 0 and regex_loop.match(linha)):
                        gravaRegistro(registro, arq)
                        registro = linha
                    else:
                        registro += linha
                    #print(repr(linha))
                    nrlinha += 1
            except UnicodeDecodeError as err:
                print(">>> Erro ao ler linhas do arquivo:",arq, "linha:", nrlinha, "Erro:", err)
                exit(0)

            #print('nr linhas', nrlinha)
                
            if len(registro) > 0:
                gravaRegistro(registro,arq)


if (len(registros_log) > 0):
    mycol.insert_many(registros_log)

Quantidade de arquivos de log: 180281
DeleteResult({'n': 77362665, 'ok': 1.0}, acknowledged=True)


100%|█████████████████████████████████████████████████████████████| 180281/180281 [15:17<00:00, 196.50it/s]


In [17]:
x = mycol.delete_many({})
print (x)

DeleteResult({'n': 3, 'ok': 1.0}, acknowledged=True)


In [6]:
result = mycol.count_documents({})
print (result)

77362665


In [30]:
result = mycol.count_documents({'arq_nome':'visibility.log'})
print (result)

24046


In [44]:
result = mycol.distinct('arq_nome')
print (result)

['Authentication.log', 'OdetteFTP.log', 'Perimeter.log', 'Security.log', 'WebSphereMQSuite.log', 'archive.log', 'bpdeadline.log', 'cccinterop.log', 'cdinterop.log', 'cdinterop_cdjava.log', 'cfgapi.log', 'cla2client.log', 'delete.log', 'ebicsClientApilogger.log', 'ebicsServerApilogger.log', 'filegateway.log', 'ftpclient.log', 'http.log', 'httpclient.log', 'jetty.log', 'mailbox.log', 'mgmtdash.log', 'noapp.log', 'ops.log', 'ops_exe.log', 'resourcemonitor.log', 'schedule.log', 'servicesctl.log', 'sftpclient.log', 'sftpserver.log', 'sftpserverlogger2_0.log', 'shutdown.log', 'socketclient.log', 'socketcommon.log', 'socketserver.log', 'sql.log', 'sshdcommonlogger.log', 'system.log', 'tracking.log', 'tx.log', 'ui.log', 'userexit.log', 'visibility.log', 'wf.log', 'wfexception.log', 'wsrm.log']


In [45]:
len(result)

46

In [8]:
result = mycol.aggregate([{"$group": {"_id": "$arq_nome", "count":{"$sum": 1}}}])
print(result)

In [11]:
count = mycol.count_documents({ 'arq_nome': "filegateway.log" })
print(f'qtd registros system_log: {count}')

qtd registros system_log: 21755


In [ ]:
## Exemplo de query com intervalo de data e limitando o resultado

import datetime 

arq = 'filegateway.log'

dt1 = datetime.datetime(2024, 4, 1, 0, 0, 0)
dt2 = datetime.datetime(2024, 4, 1, 23, 59, 59)
print(dt1,dt2)
count = mycol.count_documents({ 'arq_nome': arq, 
                               'timestamp':
                                            {"$gt": dt1, "$lt": dt2}})
print(f'qtd registros em {arq}: {count}')

c1 = mycol.find({'arq_nome': arq, 
                 'timestamp':{"$gt": dt1, "$lt": dt2}}).limit(3)

all_results = list(c1)
for document in all_results:
    print(document['texto'])



In [ ]:
## exemplo de query simples

from pymongo import MongoClient

# Replace the placeholder with your connection string.
uri = "mongodb://localhost:27017/"
client = MongoClient(uri)

try:
    agg_db = client["logdb"]

    logs_coll = agg_db["logs"]
    
    pipeline = []

    pipeline.append( {'$match': {'arq_nome': 'system.log'} })

    pipeline.append( {'$sort': {"timestamp": -1} })

    pipeline.append( {'$limit': 3})

    pipeline.append( {"$unset": ["_id","texto"]})

    aggregation_result = logs_coll.aggregate(pipeline)

    # Print the aggregation results.
    for document in aggregation_result:
        print(document)

finally:
    client.close()


In [1]:
## exemplo de agregacao
# recupera menor e maior data e quantidade total de registros de log.

from pymongo import MongoClient

# Replace the placeholder with your connection string.
uri = "mongodb://localhost:27017/"
client = MongoClient(uri)

try:
    agg_db = client["logdb"]

    logs_coll = agg_db["logs"]
    
    pipeline = []

    pipeline.append({
        "$group": {
            "_id": "",
            "primeira data": {"$min": "$timestamp"},
            "ultima data": {"$max": "$timestamp"},
            "qtd": {"$sum": 1},
        }
    })    

    pipeline.append( {'$sort': {"qtd": -1} })

    aggregation_result = logs_coll.aggregate(pipeline)

    # Print the aggregation results.
    for document in aggregation_result:
        print(document)

finally:
    client.close()


{'_id': 'arq_nome', 'primeira data': datetime.datetime(2023, 9, 29, 8, 1, 0, 892000), 'ultima data': datetime.datetime(2024, 4, 4, 8, 31, 3, 928000), 'qtd': 77362665}


In [3]:
## exemplo de agregacao
# recupera menor e maior data e quantidade total de registros de log, agrupado por identificador de log

from pymongo import MongoClient

# Replace the placeholder with your connection string.
uri = "mongodb://localhost:27017/"
client = MongoClient(uri)

try:
    agg_db = client["logdb"]

    logs_coll = agg_db["logs"]
    
    pipeline = []

    pipeline.append({
        "$group": {
            "_id": "$arq_nome",
            "primeira data": {"$min": "$timestamp"},
            "ultima data": {"$max": "$timestamp"},
            "qtd": {"$sum": 1},
        }
    })      

    pipeline.append( {'$sort': {"qtd": -1} })

    aggregation_result = logs_coll.aggregate(pipeline)

    # Print the aggregation results.
    for document in aggregation_result:
        print(document)

finally:
    client.close()


{'_id': 'cdinterop_cdjava.log', 'primeira data': datetime.datetime(2023, 9, 30, 14, 11, 38, 772000), 'ultima data': datetime.datetime(2024, 4, 4, 7, 50, 27, 951000), 'qtd': 26741718}
{'_id': 'OdetteFTP.log', 'primeira data': datetime.datetime(2023, 9, 29, 8, 3, 23, 504000), 'ultima data': datetime.datetime(2024, 4, 4, 5, 53, 57, 543000), 'qtd': 23591617}
{'_id': 'cccinterop.log', 'primeira data': datetime.datetime(2023, 9, 29, 8, 4, 52, 739000), 'ultima data': datetime.datetime(2024, 4, 4, 7, 17, 9, 403000), 'qtd': 20909562}
{'_id': 'noapp.log', 'primeira data': datetime.datetime(2023, 9, 29, 8, 1, 18, 964000), 'ultima data': datetime.datetime(2024, 4, 3, 23, 59, 41, 760000), 'qtd': 2002232}
{'_id': 'Perimeter.log', 'primeira data': datetime.datetime(2023, 9, 29, 8, 3, 21, 209000), 'ultima data': datetime.datetime(2024, 4, 3, 23, 57, 30, 730000), 'qtd': 1994356}
{'_id': 'ops.log', 'primeira data': datetime.datetime(2023, 9, 29, 8, 1, 0, 892000), 'ultima data': datetime.datetime(2024, 4